In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Подготовка данных

Загрузка данных

In [2]:
xlsx_file = pd.ExcelFile('input/БОГРАД_PISY.xlsx')
trees = ['pisy_01a', 'pisy_01b', 'pisy_02a', 'pisy_03a', 'pisy_07a', 'pisy_12b', 'pisy_14a']
columns = {_:f'D{__}' if __<16 else f'CWT{__-15}' for _, __ in zip(range(2,32), range(1,31))}
columns[0] = 'Tree'
columns[1] = 'Year'

Функция нормализации трахеид

In [3]:
def get_normalized_list(x: list, norm : int):
    """
    Функция получения нормированного списка
    :param x: список для нормирования
    :param norm: норма
    :return: l_norm - нормированный к e список l
    """
    l_raw = []  # промежуточный список
    n = len(x)
    for i in range(n):
        for j in range(norm):
            l_raw += [x[i]]
    l_norm = []
    for i in range(norm):
        l_norm += [1 / n * sum([l_raw[j] for j in range(n * i, n * (i + 1))])]
    return l_norm

Нормализуем входные данные

In [4]:
dataframes = []
for tree in trees:
    df = xlsx_file.parse(tree)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df = df.dropna(axis=0)

    norm_traches = dict()
    for year in set(df['Год']):
        norm_traches[int(year)] = [tree, int(year)] + get_normalized_list(list(df[df['Год']==year]['Dmean']), 15)+ get_normalized_list(list(df[df['Год']==year]['CWTmean']), 15)
    
    dataframes += [pd.DataFrame(norm_traches).transpose().rename(columns=columns).reset_index(drop=True)]

df = pd.concat(dataframes).reset_index(drop=True)

Сохраняем их в .xlsx файл

In [5]:
df.to_excel('output/Bograd_PISY_normalized.xlsx', index=False)

Рассчёт средних значений трахеид по годам и по деревьям:

In [6]:
mean_objects_years = dict()

for year in set(df['Year']):
    temp_data = df[df['Year']==year]
    if len(temp_data) > 3:
        mean_objects_years[year] = temp_data.mean()[1:]


mean_objects_trees = dict()

for tree in set(df['Tree']):
    mean_objects_trees[tree] = df[df['Tree']==tree].mean()[1:]

Считаем средние значения трахеид по всем записям

In [7]:
global_mean = df.mean()[1:]

Строим таблицы объектов для метода A -- кластеризация отклонений средних объектов по году от среднего глобального объекта:

In [8]:
quotient_deviation_df_A = []
difference_deviation_df_A = []

_columns = {_:f'D{_}' if _<16 else f'CWT{_-15}' for _ in  range(1,31)}
_columns[0] = 'Year'

for year, mean_obj in mean_objects_years.items():
    quotient_deviation_df_A += [[year] + list(mean_obj/global_mean)]
    difference_deviation_df_A += [[year] + list(mean_obj-global_mean)]

quotient_deviation_df_A = pd.DataFrame(quotient_deviation_df_A).rename(columns=_columns)
difference_deviation_df_A = pd.DataFrame(difference_deviation_df_A).rename(columns=_columns)

In [9]:
quotient_deviation_df_A.to_excel('output/quotient_deviation_df_A.xlsx', index=False)
difference_deviation_df_A.to_excel('output/difference_deviation_df_A.xlsx', index=False)

Строим таблицы объектов для метода B -- кластеризация средних отклонений объектов по году от среднего объекта по дереву:

In [10]:
qd_df_B = []
dd_df_B = []

quotient_deviation_df_B = dict()
difference_deviation_df_B = dict()


for _, row in df.iterrows():
    qd_df_B += [[row[0], row[1]] + list(row[2:] / mean_objects_trees[row[0]])]
    dd_df_B +=  [[row[0], row[1]] + list(row[2:] - mean_objects_trees[row[0]])]

qd_df_B = pd.DataFrame(qd_df_B).rename(columns=columns)
dd_df_B = pd.DataFrame(dd_df_B).rename(columns=columns)

for year in set(df['Year']):
    temp_data_q = qd_df_B[qd_df_B['Year']==year]
    temp_data_d = dd_df_B[dd_df_B['Year']==year]
    if len(temp_data_q) > 3:
        quotient_deviation_df_B[year] = temp_data_q.mean()[1:]
        difference_deviation_df_B[year] = temp_data_d.mean()[1:]

quotient_deviation_df_B = pd.DataFrame(quotient_deviation_df_B).transpose()
difference_deviation_df_B = pd.DataFrame(difference_deviation_df_B).transpose()

In [11]:
quotient_deviation_df_B.to_excel('output/quotient_deviation_df_B.xlsx', index=True)
difference_deviation_df_B.to_excel('output/difference_deviation_df_B.xlsx', index=True)

# Кластеризация

In [12]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


def get_models(data, name='Data'):
    models = []
    inertia = []
    silhouette = []
    print(name, end=': ')
    for n_clusters in range(2, 16):
        print(n_clusters, end=', ')
        # Описываем модель
        model = KMeans(n_clusters=n_clusters, max_iter=5000, random_state=0)

        # Проводим моделирование
        model.fit(data)

        # Предсказание на всем наборе данных
        all_predictions = model.predict(data)

        # Распихиваем точки по кластерам
        clusters = [[] for i in range(n_clusters)]
        for i, num in enumerate(all_predictions):
            clusters[num] += [data[i]]
        
        models += [model]
        inertia += [model.inertia_]
        silhouette += [silhouette_score(data, model.labels_, metric='euclidean')]
    print('done!')
    return models, inertia, silhouette

Тут формируем массивы обучабщих выборок для обоих методов:

In [13]:
quo_data_A  = np.array(quotient_deviation_df_A.drop(['Year'], axis=1))
diff_data_A = np.array(difference_deviation_df_A.drop(['Year'], axis=1))
quo_data_B  = np.array(quotient_deviation_df_B)
diff_data_B = np.array(difference_deviation_df_B)

Обучаем модели для каждого метода:

In [14]:
models_quo_A, i_1, s_1 = get_models(quo_data_A, 'A quo')
models_diff_A, i_2, s_2 = get_models(diff_data_A, 'A diff')
models_quo_B, i_1, s_1 = get_models(quo_data_B, 'B quo')
models_diff_B, i_2, s_2 = get_models(diff_data_B, 'B diff')

A quo: 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, done!
A diff: 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, done!
B quo: 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, done!
B diff: 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, done!


Функция сохранения графиков кластеров и их средних объектов:

In [15]:
def save_clusters(models, data, amount=15, num=3, data_type='below', ylim=None):
    all_predictions = models[num-2].predict(data)
    try:
        os.mkdir(f'output/{data_type}/')
    except Exception as e:
        print(e)
    try:
        os.mkdir(f'output/{data_type}/{num}_clust/')
    except Exception as e:
        print(e)
    for j in range(num):
        fig, ax = plt.subplots( nrows=1, ncols=1)
        ax.plot(range(1,amount+1), models[num-2].cluster_centers_[j])
        ax.set_title(f'Mean object (cluster №{j+1})')
        if ylim:
            ax.set_ylim(ylim)
        fig.savefig(f'output/{data_type}/{num}_clust/mean_obj_cluster_{j+1}.png')
        plt.close(fig)
        
        fig, ax = plt.subplots( nrows=1, ncols=1)
        els = 0
        for i in range(len(all_predictions)):
            if all_predictions[i] == j:
                ax.plot(range(1,amount+1), data[i], label=str(i))
                els += 1
        #legend(frameon=False)
        if ylim:
            ax.set_ylim(ylim)
        ax.set_title(f'Cluster №{j+1} ({els} elements)')
        fig.savefig(f'output/{data_type}/{num}_clust/cluster_{j+1}.png') 
        plt.close(fig)

Сохраняем графики для всех моделей:

In [16]:
for _ in range(3, 6):
    save_clusters(models_quo_A, quo_data_A, 30, _, 'A_Quotient', [0.5, 1.5])
    save_clusters(models_diff_A, diff_data_A, 30, _, 'A_Difference', ylim=[-15, 15])
    save_clusters(models_quo_B, quo_data_B, 30, _, 'B_Quotient', [0.5, 1.5])
    save_clusters(models_diff_B, diff_data_B, 30, _, 'B_Difference', ylim=[-15, 15])

[WinError 183] Невозможно создать файл, так как он уже существует: 'output/A_Quotient/'
[WinError 183] Невозможно создать файл, так как он уже существует: 'output/A_Difference/'
[WinError 183] Невозможно создать файл, так как он уже существует: 'output/B_Quotient/'
[WinError 183] Невозможно создать файл, так как он уже существует: 'output/B_Difference/'
[WinError 183] Невозможно создать файл, так как он уже существует: 'output/A_Quotient/'
[WinError 183] Невозможно создать файл, так как он уже существует: 'output/A_Difference/'
[WinError 183] Невозможно создать файл, так как он уже существует: 'output/B_Quotient/'
[WinError 183] Невозможно создать файл, так как он уже существует: 'output/B_Difference/'


Сохраняем таблицы кластеризованных объектов:

In [17]:
quotient_deviation_df_A['Class 3'] = models_quo_A[1].predict(quo_data_A)
quotient_deviation_df_A['Class 4'] = models_quo_A[2].predict(quo_data_A)
quotient_deviation_df_A['Class 5'] = models_quo_A[3].predict(quo_data_A)
quotient_deviation_df_A.to_excel('output/quotient_deviation_df_A_CLASSIFIED.xlsx', index=False)

difference_deviation_df_A['Class 3'] = models_diff_A[1].predict(diff_data_A)
difference_deviation_df_A['Class 4'] = models_diff_A[2].predict(diff_data_A)
difference_deviation_df_A['Class 5'] = models_diff_A[3].predict(diff_data_A)
difference_deviation_df_A.to_excel('output/difference_deviation_df_A_CLASSIFIED.xlsx', index=False)

In [18]:
quotient_deviation_df_B['Class 3'] = models_quo_B[1].predict(quo_data_B)
quotient_deviation_df_B['Class 4'] = models_quo_B[2].predict(quo_data_B)
quotient_deviation_df_B['Class 5'] = models_quo_B[3].predict(quo_data_B)
quotient_deviation_df_B.to_excel('output/quotient_deviation_df_B_CLASSIFIED.xlsx', index=True)

difference_deviation_df_B['Class 3'] = models_diff_B[1].predict(diff_data_B)
difference_deviation_df_B['Class 4'] = models_diff_B[2].predict(diff_data_B)
difference_deviation_df_B['Class 5'] = models_diff_B[3].predict(diff_data_B)
difference_deviation_df_B.to_excel('output/difference_deviation_df_B_CLASSIFIED.xlsx', index=True)

# Сравнение методов:

In [19]:
import numpy as np
from scipy.stats import pearsonr

def dropna_pearsonr(x, y):
    x, y = np.array(x), np.array(y)
    nas = np.logical_or(np.isnan(x), np.isnan(y))
    x, y = x[~nas], y[~nas]
    r, p = pearsonr(x, y)
    return r, p
